# Hominid dataset analysis

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [17]:
#read data
hominid = pd.read_csv("Homininos_DataSet (1).csv")
print(hominid.head())

                       Genus_&_Specie        Time Location            Zone  \
0          Australopithecus Anamensis   4.2- 3.9    Africa        oriental   
1  hominino Sahelanthropus tchadensis           7   Africa         central   
2          Australopithecus Afarensis  3.9 - 2.9    Africa        oriental   
3  hominino Sahelanthropus tchadensis           7   Africa         central   
4                Homo Heidelbergensis     500-250   Europa  central-europe   

    Current_Country         Habitat  Cranial_Capacity  Height Incisor_Size  \
0             Kenya  forest-gallery            281.19  132.33          big   
1  Republic of Chad           mixed             94.12   87.49        small   
2             Kenya           mixed            311.48  120.44          big   
3  Republic of Chad           mixed            196.00   88.40        small   
4           Germany     cold forest           1232.46  167.60        small   

  Jaw_Shape  ...            biped        Arms     Foots       

The column 'Time' is crucial for my analyses but the formatting is not consistent among all data points and cannot be interpreted as an ordered categorical variable. In the following lines I develop a loop in which I

1.Go through each row
2.Check if the line contains a '-'
3.If the line contains a '-', split the line and keep only the first number
4.Check if the number contains a 'ma'
5.If the number contains a 'ma', keep only the first character
6.If the number doesn't contain a 'ma', keep only the first chacter and add a '0.' before it

The previous part of the loop ensures I only keep the oldest known appearance of the species, expressed as million years ago, and uses 

In [30]:
hominid.Time.describe()


0       4.2- 3.9 
1               7
2      3.9 - 2.9 
3               7
4         500-250
          ...    
715       1ma-800
716           2.6
717       1ma-800
718    3.9 - 2.9 
719      2.5-1.6 
Name: Time, Length: 720, dtype: object